In [24]:

import numpy as np  
import pandas as pd 
import os 
import re
import warnings
from tqdm import tqdm
import datetime as dt

# ML/DL imports
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras

# Plotting imports
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

from numpy.random import seed
seed(1)

# Allows us to see more information regarding the DataFrame
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)


In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [3]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

# Reading in the datasets

In [3]:
path = '../datasets/m5-forecasting-accuracy/'

In [4]:

#train_sales = pd.read_csv(path+'sales_train_validation.csv')
calendar = pd.read_csv(path+'calendar.csv')
sell_prices = pd.read_csv(path+'sell_prices.csv')
df_val = pd.read_csv(path+'sales_train_evaluation.csv')

In [5]:
train_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


In [6]:
downcast_dtypes(train_sales)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1

In [7]:
#downcast_dtypes(calendar)
downcast_dtypes(sell_prices)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [8]:
downcast_dtypes(calendar)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [7]:
train_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int16(1913), object(6)
memory usage: 112.6+ MB


# Preparing the Data

In [4]:
train_sales.describe()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
count,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30

In [5]:
# d_1, d_2, …, d_i, … d_1941: The number of units sold at day i, starting from 2011-01-29.


train_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1

In [6]:
#date: The date in a “y-m-d” format.

# wm_yr_wk: The id of the week the date belongs to.

# weekday: The type of the day (Saturday, Sunday, …, Friday).

# wday: The id of the weekday, starting from Saturday.

# month: The month of the date.

# year: The year of the date.
# snap_CA, snap_TX, and snap_WI: A binary variable (0 or 1) indicating whether the stores of CA, TX or WI allow SNAP3 purchases on the examined date. 1 indicates that SNAP purchases are allowed.


calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [7]:
calendar.describe()

,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI
count,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000
mean,11347.086338,3.997461,6.325546,2013.288471,0.330117,0.330117,0.330117
std,155.277043,2.001141,3.416864,1.580198,0.470374,0.470374,0.470374
min,11101.000000,1.000000,1.000000,2011.000000,0.000000,0.000000,0.000000
25%,11219.000000,2.000000,3.000000,2012.000000,0.000000,0.000000,0.000000
50%,11337.000000,4.000000,6.000000,2013.000000,0.000000,0.000000,0.000000
75%,11502.000000,6.000000,9.000000,2015.000000,1.000000,1.000000,1.000000
max,11621.000000,7.000000,12.000000,2016.000000,1.000000,1.000000,1.000000


In [8]:
#sell_price: The price of the product for the given week/store. 

#The price is provided per week (average across seven days). If not available, this means that the product was not sold during the examined week. 
#Note that although prices are constant at weekly basis, they may change through time (both training and test set).



sell_prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [9]:
sell_prices.describe()

,wm_yr_wk,sell_price
count,6.841121e+06,6.841121e+06
mean,1.138294e+04,4.410952e+00
std,1.486100e+02,3.408814e+00
min,1.110100e+04,1.000000e-02
25%,1.124700e+04,2.180000e+00
50%,1.141100e+04,3.470000e+00
75%,1.151700e+04,5.840000e+00
max,1.162100e+04,1.073200e+02


In [1]:
calendar

NameError: name 'calendar' is not defined

In [11]:
calendar.isna().sum()

date               0
wm_yr_wk           0
weekday            0
wday               0
month              0
year               0
d                  0
event_name_1    1807
event_type_1    1807
event_name_2    1964
event_type_2    1964
snap_CA            0
snap_TX            0
snap_WI            0
dtype: int64

In [12]:
train_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


In [13]:
train_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1

In [5]:
# merging calendar and price data on week, month and year
# The price is provided per week (average across seven days)

sell_prices_calendar = pd.merge(sell_prices, calendar, on = 'wm_yr_wk')
#sell_prices_calendar.head()

In [6]:
len(sell_prices_calendar)

47735397

In [15]:
sell_prices_calendar.head()

,store_id,item_id,wm_yr_wk,sell_price,date,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,CA_1,HOBBIES_1_001,11325,9.58,2013-07-13,Saturday,1,7,2013,d_897,NaN,NaN,NaN,NaN,0,1,0
1,CA_1,HOBBIES_1_001,11325,9.58,2013-07-14,Sunday,2,7,2013,d_898,NaN,NaN,NaN,NaN,0,0,1
2,CA_1,HOBBIES_1_001,11325,9.58,2013-07-15,Monday,3,7,2013,d_899,NaN,NaN,NaN,NaN,0,1,1
3,CA_1,HOBBIES_1_001,11325,9.58,2013-07-16,Tuesday,4,7,2013,d_900,NaN,NaN,NaN,NaN,0,0,0
4,CA_1,HOBBIES_1_001,11325,9.58,2013-07-17,Wednesday,5,7,2013,d_901,NaN,NaN,NaN,NaN,0,0,0


In [7]:
reduced_sell_prices_calendar = sell_prices_calendar.drop(['wm_yr_wk','weekday','wday','month','year','event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], axis=1)

In [ ]:
len(reduced_sell_prices_calendar)

In [ ]:
#reduced_sell_prices_calendar_sort = reduced_sell_prices_calendar.sort_values(by = 'date')

In [ ]:
#reduced_sell_prices_calendar_sort

In [8]:
# changing the sales dataset to turn the 'd' columns to roll to help mearge the dataset,

#train_sales_melt = train_sales.melt(id_vars=["id","item_id","dept_id", "cat_id","store_id","state_id"], var_name="d", value_name="sold_units")
#train_sales_melt


df_val_melt = df_val.melt(id_vars=["id","item_id","dept_id", "cat_id","store_id","state_id"], var_name="d", value_name="sold_units")
df_val_melt

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1,0,1,4,0,0,5,0,0,0,0,0,0,2,1,2,1,0,0,0,1,1,1,0,0,1,1,1,1

In [9]:
# merging the 3 data datasets together
#sales_prices = pd.merge(reduced_sell_prices_calendar, train_sales_melt, on = ['item_id', 'store_id', 'd'])
#sales_prices.head()

val_sales_prices = pd.merge(reduced_sell_prices_calendar, df_val_melt, on = ['item_id', 'store_id', 'd'])
val_sales_prices

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold_units
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0


In [10]:
val_sales_prices

,store_id,item_id,sell_price,date,d,snap_CA,snap_TX,snap_WI,id,dept_id,cat_id,state_id,sold_units
0,CA_1,HOBBIES_1_001,9.58,2013-07-13,d_897,0,1,0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0
1,CA_1,HOBBIES_1_001,9.58,2013-07-14,d_898,0,0,1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0
2,CA_1,HOBBIES_1_001,9.58,2013-07-15,d_899,0,1,1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0
3,CA_1,HOBBIES_1_001,9.58,2013-07-16,d_900,0,0,0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0
4,CA_1,HOBBIES_1_001,9.58,2013-07-17,d_901,0,0,0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,WI_3,FOODS_3_825,4.00,2011-02-28,d_31,0,0,0,FOODS_3_825_WI_3_evaluation,FOODS_3,FOODS,WI,1
46881673,WI_3,FOODS_3_825,4.00,2011-03-01,d_32,1,1,0,FOODS_3_825_WI_3_evaluation,FOODS_3,FOODS,WI,2
46881674,WI_3,FOODS_3_825,4.00,2011-03-02,d_33,1,0,1,FOODS_3_825_WI_3_evaluation,FOODS_3,FOODS,WI,2
46881675,WI_3,FOODS_3_825,4.00,2011-03-03,d_34,1,1,1,FOODS_3_825_WI_3_evaluation,FOODS_3,FOODS,WI,2


In [11]:
# to convert the sell_price and sold_units to floats
#sales_prices[['sell_price', 'sold_units']] = sales_prices[['sell_price', 'sold_units']].astype(float)



val_sales_prices[['sell_price', 'sold_units']] = val_sales_prices[['sell_price', 'sold_units']].astype(float)
val_sales_prices

,store_id,item_id,sell_price,date,d,snap_CA,snap_TX,snap_WI,id,dept_id,cat_id,state_id,sold_units
0,CA_1,HOBBIES_1_001,9.58,2013-07-13,d_897,0,1,0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0
1,CA_1,HOBBIES_1_001,9.58,2013-07-14,d_898,0,0,1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0
2,CA_1,HOBBIES_1_001,9.58,2013-07-15,d_899,0,1,1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0
3,CA_1,HOBBIES_1_001,9.58,2013-07-16,d_900,0,0,0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0
4,CA_1,HOBBIES_1_001,9.58,2013-07-17,d_901,0,0,0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,WI_3,FOODS_3_825,4.00,2011-02-28,d_31,0,0,0,FOODS_3_825_WI_3_evaluation,FOODS_3,FOODS,WI,1.0
46881673,WI_3,FOODS_3_825,4.00,2011-03-01,d_32,1,1,0,FOODS_3_825_WI_3_evaluation,FOODS_3,FOODS,WI,2.0
46881674,WI_3,FOODS_3_825,4.00,2011-03-02,d_33,1,0,1,FOODS_3_825_WI_3_evaluation,FOODS_3,FOODS,WI,2.0
46881675,WI_3,FOODS_3_825,4.00,2011-03-03,d_34,1,1,1,FOODS_3_825_WI_3_evaluation,FOODS_3,FOODS,WI,2.0


In [14]:
# creating a new column to calculate revenue
#sales_prices['revenue'] = sales_prices['sell_price'] * sales_prices['sold_units']
#sales_prices

In [12]:
#Sales_prices_index = sales_prices.set_index('date', inplace=True)

val_sales_prices.set_index('date', inplace=True)

In [13]:
val_sales_prices.head(10)

,store_id,item_id,sell_price,d,snap_CA,snap_TX,snap_WI,id,dept_id,cat_id,state_id,sold_units
date,,,,,,,,,,,,
2013-07-13,CA_1,HOBBIES_1_001,9.58,d_897,0,1,0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0
2013-07-14,CA_1,HOBBIES_1_001,9.58,d_898,0,0,1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0
2013-07-15,CA_1,HOBBIES_1_001,9.58,d_899,0,1,1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0
2013-07-16,CA_1,HOBBIES_1_001,9.58,d_900,0,0,0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0
2013-07-17,CA_1,HOBBIES_1_001,9.58,d_901,0,0,0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0
2013-07-18,CA_1,HOBBIES_1_001,9.58,d_902,0,0,0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,1.0
2013-07-19,CA_1,HOBBIES_1_001,9.58,d_903,0,0,0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0
2013-07-13,CA_1,HOBBIES_1_002,3.97,d_897,0,1,0,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,1.0
2013-07-14,CA_1,HOBBIES_1_002,3.97,d_898,0,0,1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0


In [14]:
#sales_prices_sort = sales_prices.sort_values(by = 'date')

sales_prices_sort = val_sales_prices.sort_values(by = 'date')



In [15]:
sales_prices_sort.head(30)

,store_id,item_id,sell_price,d,snap_CA,snap_TX,snap_WI,id,dept_id,cat_id,state_id,sold_units
date,,,,,,,,,,,,
2011-01-29,CA_4,HOUSEHOLD_2_028,5.94,d_1,0,0,0,HOUSEHOLD_2_028_CA_4_evaluation,HOUSEHOLD_2,HOUSEHOLD,CA,0.0
2011-01-29,TX_2,FOODS_3_591,2.68,d_1,0,0,0,FOODS_3_591_TX_2_evaluation,FOODS_3,FOODS,TX,0.0
2011-01-29,CA_3,HOUSEHOLD_2_488,4.47,d_1,0,0,0,HOUSEHOLD_2_488_CA_3_evaluation,HOUSEHOLD_2,HOUSEHOLD,CA,1.0
2011-01-29,TX_1,FOODS_3_686,3.50,d_1,0,0,0,FOODS_3_686_TX_1_evaluation,FOODS_3,FOODS,TX,0.0
2011-01-29,WI_3,FOODS_3_252,1.48,d_1,0,0,0,FOODS_3_252_WI_3_evaluation,FOODS_3,FOODS,WI,22.0
2011-01-29,TX_3,FOODS_2_394,2.28,d_1,0,0,0,FOODS_2_394_TX_3_evaluation,FOODS_2,FOODS,TX,0.0
2011-01-29,CA_1,HOUSEHOLD_1_268,7.97,d_1,0,0,0,HOUSEHOLD_1_268_CA_1_evaluation,HOUSEHOLD_1,HOUSEHOLD,CA,0.0
2011-01-29,CA_1,HOBBIES_1_422,2.87,d_1,0,0,0,HOBBIES_1_422_CA_1_evaluation,HOBBIES_1,HOBBIES,CA,0.0
2011-01-29,WI_3,FOODS_3_783,5.98,d_1,0,0,0,FOODS_3_783_WI_3_evaluation,FOODS_3,FOODS,WI,1.0


In [16]:
sales_prices_sort.tail(30)

,store_id,item_id,sell_price,d,snap_CA,snap_TX,snap_WI,id,dept_id,cat_id,state_id,sold_units
date,,,,,,,,,,,,
2016-05-22,TX_1,HOUSEHOLD_2_276,5.47,d_1941,0,0,0,HOUSEHOLD_2_276_TX_1_evaluation,HOUSEHOLD_2,HOUSEHOLD,TX,0.0
2016-05-22,TX_1,HOUSEHOLD_2_278,10.98,d_1941,0,0,0,HOUSEHOLD_2_278_TX_1_evaluation,HOUSEHOLD_2,HOUSEHOLD,TX,0.0
2016-05-22,TX_1,HOUSEHOLD_2_279,4.44,d_1941,0,0,0,HOUSEHOLD_2_279_TX_1_evaluation,HOUSEHOLD_2,HOUSEHOLD,TX,0.0
2016-05-22,TX_1,HOUSEHOLD_2_280,5.97,d_1941,0,0,0,HOUSEHOLD_2_280_TX_1_evaluation,HOUSEHOLD_2,HOUSEHOLD,TX,0.0
2016-05-22,TX_1,HOUSEHOLD_2_281,5.27,d_1941,0,0,0,HOUSEHOLD_2_281_TX_1_evaluation,HOUSEHOLD_2,HOUSEHOLD,TX,1.0
2016-05-22,TX_1,HOUSEHOLD_2_282,5.84,d_1941,0,0,0,HOUSEHOLD_2_282_TX_1_evaluation,HOUSEHOLD_2,HOUSEHOLD,TX,0.0
2016-05-22,TX_1,HOUSEHOLD_2_283,5.84,d_1941,0,0,0,HOUSEHOLD_2_283_TX_1_evaluation,HOUSEHOLD_2,HOUSEHOLD,TX,0.0
2016-05-22,TX_1,HOUSEHOLD_2_284,5.94,d_1941,0,0,0,HOUSEHOLD_2_284_TX_1_evaluation,HOUSEHOLD_2,HOUSEHOLD,TX,1.0
2016-05-22,TX_1,HOUSEHOLD_2_277,3.33,d_1941,0,0,0,HOUSEHOLD_2_277_TX_1_evaluation,HOUSEHOLD_2,HOUSEHOLD,TX,0.0


In [23]:
sales_prices.to_csv('sales_prices.csv') 

In [18]:
val_sales_prices.to_csv('val_sales_prices.csv') 

# Data Preparation to extract validation subset

In [5]:
val_sales_prices = pd.read_csv('val_sales_prices.csv')

In [6]:
val_sales_prices.drop(['id', 'dept_id', 'item_id', 'd', 'state_id',], axis=1, inplace=True)

In [7]:
sales_prices_cleaned = downcast_dtypes(val_sales_prices)

In [8]:
sales_prices_cleaned

,date,store_id,sell_price,snap_CA,snap_TX,snap_WI,cat_id,sold_units
0,2013-07-13,CA_1,9.58,0,1,0,HOBBIES,0.0
1,2013-07-14,CA_1,9.58,0,0,1,HOBBIES,0.0
2,2013-07-15,CA_1,9.58,0,1,1,HOBBIES,0.0
3,2013-07-16,CA_1,9.58,0,0,0,HOBBIES,0.0
4,2013-07-17,CA_1,9.58,0,0,0,HOBBIES,0.0
...,...,...,...,...,...,...,...,...
46881672,2011-02-28,WI_3,4.00,0,0,0,FOODS,1.0
46881673,2011-03-01,WI_3,4.00,1,1,0,FOODS,2.0
46881674,2011-03-02,WI_3,4.00,1,0,1,FOODS,2.0
46881675,2011-03-03,WI_3,4.00,1,1,1,FOODS,2.0


In [9]:
sales_prices_cleaned_grouped = sales_prices_cleaned.groupby(['date', 'store_id', 'cat_id']).agg({'sold_units': 'sum'}).reset_index()

In [10]:
sales_prices_cleaned_grouped.shape

(58230, 4)

In [11]:
sales_prices_cleaned_grouped['store_cat_id'] = sales_prices_cleaned_grouped[['store_id', 'cat_id']].apply(lambda x: '_'.join(x), axis=1)

In [12]:
sales_prices_cleaned_grouped.head()

,date,store_id,cat_id,sold_units,store_cat_id
0,2011-01-29,CA_1,FOODS,3239.0,CA_1_FOODS
1,2011-01-29,CA_1,HOBBIES,556.0,CA_1_HOBBIES
2,2011-01-29,CA_1,HOUSEHOLD,542.0,CA_1_HOUSEHOLD
3,2011-01-29,CA_2,FOODS,2193.0,CA_2_FOODS
4,2011-01-29,CA_2,HOBBIES,538.0,CA_2_HOBBIES


,date,sold_units,store_cat_id
0,2011-01-29,3239.0,CA_1_FOODS
1,2011-01-29,556.0,CA_1_HOBBIES
2,2011-01-29,542.0,CA_1_HOUSEHOLD
3,2011-01-29,2193.0,CA_2_FOODS
4,2011-01-29,538.0,CA_2_HOBBIES
...,...,...,...
58225,2016-05-22,281.0,WI_2_HOBBIES
58226,2016-05-22,1230.0,WI_2_HOUSEHOLD
58227,2016-05-22,3484.0,WI_3_FOODS
58228,2016-05-22,342.0,WI_3_HOBBIES


In [16]:
sales_prices_cleaned_grouped.drop(['store_id', 'cat_id'], inplace=True, axis=1)

In [17]:
sales_prices_cleaned_grouped

,date,sold_units,store_cat_id
0,2011-01-29,3239.0,CA_1_FOODS
1,2011-01-29,556.0,CA_1_HOBBIES
2,2011-01-29,542.0,CA_1_HOUSEHOLD
3,2011-01-29,2193.0,CA_2_FOODS
4,2011-01-29,538.0,CA_2_HOBBIES
...,...,...,...
58225,2016-05-22,281.0,WI_2_HOBBIES
58226,2016-05-22,1230.0,WI_2_HOUSEHOLD
58227,2016-05-22,3484.0,WI_3_FOODS
58228,2016-05-22,342.0,WI_3_HOBBIES


In [18]:
final_store_cat_df = (
    pd.pivot_table(sales_prices_cleaned_grouped, values="sold_units", columns=['store_cat_id'], index=['date'])
    .reset_index()
)

In [19]:
val_final_store_cat_df = final_store_cat_df.to_csv('val_final_store_cat_df.csv')

In [27]:
val_final_store_cat_df = pd.read_csv('val_final_store_cat_df.csv')

In [28]:
val_final_store_cat_df

,Unnamed: 0,date,CA_1_FOODS,CA_1_HOBBIES,CA_1_HOUSEHOLD,CA_2_FOODS,CA_2_HOBBIES,CA_2_HOUSEHOLD,CA_3_FOODS,CA_3_HOBBIES,CA_3_HOUSEHOLD,CA_4_FOODS,CA_4_HOBBIES,CA_4_HOUSEHOLD,TX_1_FOODS,TX_1_HOBBIES,TX_1_HOUSEHOLD,TX_2_FOODS,TX_2_HOBBIES,TX_2_HOUSEHOLD,TX_3_FOODS,TX_3_HOBBIES,TX_3_HOUSEHOLD,WI_1_FOODS,WI_1_HOBBIES,WI_1_HOUSEHOLD,WI_2_FOODS,WI_2_HOBBIES,WI_2_HOUSEHOLD,WI_3_FOODS,WI_3_HOBBIES,WI_3_HOUSEHOLD
0,0,2011-01-29,3239.0,556.0,542.0,2193.0,538.0,763.0,3446.0,550.0,743.0,1223.0,158.0,244.0,1781.0,251.0,524.0,2788.0,385.0,679.0,2284.0,243.0,503.0,1581.0,615.0,508.0,1615.0,190.0,451.0,3028.0,278.0,732.0
1,1,2011-01-30,3137.0,498.0,520.0,1921.0,397.0,728.0,3535.0,430.0,862.0,1269.0,236.0,272.0,1903.0,264.0,520.0,2870.0,359.0,708.0,2257.0,247.0,502.0,1327.0,443.0,424.0,1433.0,127.0,362.0,3106.0,356.0,736.0
2,2,2011-01-31,2008.0,415.0,393.0,1289.0,368.0,464.0,2701.0,438.0,646.0,946.0,251.0,189.0,1377.0,107.0,338.0,2082.0,229.0,420.0,1665.0,190.0,370.0,977.0,323.0,262.0,1586.0,113.0,319.0,2543.0,248.0,526.0
3,3,2011-02-01,2258.0,392.0,401.0,1540.0,350.0,434.0,3064.0,424.0,744.0,1002.0,239.0,199.0,1643.0,248.0,367.0,2180.0,359.0,415.0,1647.0,202.0,320.0,935.0,137.0,179.0,2013.0,124.0,385.0,2596.0,194.0,421.0
4,4,2011-02-02,2032.0,268.0,330.0,1278.0,296.0,368.0,2761.0,364.0,692.0,1107.0,253.0,176.0,1329.0,102.0,263.0,1890.0,257.0,345.0,1383.0,142.0,201.0,2.0,0.0,0.0,967.0,58.0,150.0,1854.0,74.0,204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,1936,2016-05-18,2816.0,381.0,798.0,2561.0,334.0,894.0,3425.0,565.0,1385.0,1663.0,326.0,530.0,1924.0,257.0,966.0,2270.0,385.0,863.0,2402.0,414.0,894.0,2283.0,373.0,586.0,3129.0,373.0,1031.0,2317.0,332.0,619.0
1937,1937,2016-05-19,2854.0,406.0,876.0,2725.0,357.0,955.0,3456.0,596.0,1528.0,1615.0,376.0,553.0,1764.0,229.0,755.0,2114.0,323.0,689.0,2205.0,278.0,805.0,2410.0,360.0,708.0,3198.0,284.0,1146.0,2396.0,266.0,736.0
1938,1938,2016-05-20,2993.0,539.0,901.0,3179.0,434.0,1138.0,3497.0,571.0,1474.0,1685.0,431.0,588.0,2384.0,281.0,999.0,2737.0,442.0,1070.0,2844.0,409.0,1137.0,2616.0,438.0,759.0,3338.0,329.0,1213.0,2863.0,269.0,994.0
1939,1939,2016-05-21,3783.0,719.0,1262.0,4502.0,762.0,1856.0,4315.0,675.0,2083.0,1889.0,545.0,712.0,2604.0,386.0,1177.0,3185.0,555.0,1062.0,3045.0,499.0,1168.0,3395.0,564.0,1043.0,3607.0,324.0,1282.0,3274.0,304.0,941.0
